In [101]:
# read in allowed_guesses and answer_list dictionaries from wordle
answer_list = [line.rstrip() for line in open('wordle_answers.txt', 'r')]
allowed_guesses = sorted([line.rstrip() for line in open('wordle_allowed_guesses.txt', 'r')] + answer_list)

In [102]:
# gives individual guess a score value with colors represented by 0, .5, and 1
from collections import Counter
def color_and_score_guess(guess, target):
    colors = []
    target_letter_counts = dict(Counter(target))
    letter_counts_guess = dict.fromkeys(list(set(target)), 0)
    for position, letter in enumerate(guess):
        if target[position] == letter and letter_counts_guess[letter] < target_letter_counts[letter]:
            colors.append(1)
            letter_counts_guess[letter] += 1
        elif letter in target and letter_counts_guess[letter] < target_letter_counts[letter]:
            colors.append(.5)
            letter_counts_guess[letter] += 1
        else:
            colors.append(0)
    score = sum(colors)
    return colors, score

def get_color(guess, target):
    return color_and_score_guess(guess, target)[0]

def get_score(guess, target):
    return color_and_score_guess(guess, target)[1]

# returns sum of scores for input word on each word in answer_list
def word_goodness(word):
    goodness = sum([get_score(word, target) for target in answer_list])
    return goodness

In [103]:
# returns goodness of each word in allowed_guesses and stores the ranking in word_ranks.csv
hi = '''def rank_all_words():
    ranks = [[word, 0] for word in allowed_guesses]
    for word_num in range(len(ranks)):
        ranks[word_num][1] += word_goodness(ranks[word_num][0])
    return ranks
    
word_ranks = sorted(rank_all_words(), key = lambda x: x[1], reverse = True)

import csv
with open("word_ranks.csv", "w", newline='') as f:
    wr = csv.writer(f)
    wr.writerows(word_ranks)'''

In [104]:
# narrows down valid_guesses based on color and returns new valid_guesses
def eliminate(valid_guesses, letter_colors):
    
    # iterates through letter_colors tuples
    for position, (letter, color) in enumerate(letter_colors):
        
        # remove valid guesses without known greens
        if color == 1:
            valid_guesses = [word for word in valid_guesses if word[position] == letter]
        
        # remove valid guesses without known yellows
        if color == .5:
            valid_guesses = [word for word in valid_guesses if letter in word]
        
        # remove valid guesses with known yellows in the yellow position
        if color == .5:
            valid_guesses = [word for word in valid_guesses if word[position] != letter]
        
        # remove valid guesses with letters that are entirely gray
        if color == 0 and (letter, .5) not in letter_colors and (letter, 1) not in letter_colors:
            valid_guesses = [word for word in valid_guesses if letter not in word]
    
    return valid_guesses

In [105]:
# WORDLE SOLVER:
# auto mode attempts to solve a given word
# cheat mode if you want to play along with an actual wordle
# play mode if you just want to play wordle

import random 
def solver(mode):
    
    # orders the words according to rank
    ordered_words = []
    with open("word_ranks.csv", "r", newline = "") as f:
        r = csv.reader(f, delimiter=",")
        for row in r:
            ordered_words.append(row[0])
    
    valid_guesses = ordered_words
    
    if mode == "auto":
        target = input("What word would you like me to solve?: ")
        print()
        # iterates through guesses
        for turn in range(1, 7):
            print("I guessed " + valid_guesses[0])
            print()
            if get_score(valid_guesses[0], target) == 5:
                print("The word is " + valid_guesses[0].upper() + ", solved in " + str(turn) + " guesses")
                return
            guess_letters = [letter for letter in valid_guesses[0]]
            guess_colors = get_color(valid_guesses[0], target)
            letter_colors = list(zip(guess_letters, guess_colors))
            # reduce valid guess size
            valid_guesses = eliminate(valid_guesses, letter_colors)
        print("Failed to solve in 6 turns")
        print()
        print("Was it any of these words?: " + str(valid_guesses[0:5]))
        return
    
    if mode == "cheat":
        count = 1
        # iterate through user prompts
        while True:
            print()
            guess = input("GUESS " + str(count) + ", RECOMMENDED: " + str(valid_guesses[0:5]) + ", ENTER YOUR GUESS: ")
            guess_letters = [letter for letter in guess]
            guess_colors = []
            for _ in range(0, 5):
                color = float(input("What are the colors? 1 for Green, .5 for Yellow, 0 for Gray: "))
                guess_colors.append(color)
            letter_colors = list(zip(guess_letters, guess_colors))
            if guess_colors == [1, 1, 1, 1, 1]:
                print()
                print("Congrats you won in " + str(count) + " guesses")
                return
            # reduce valid guess size
            valid_guesses = eliminate(valid_guesses, letter_colors)
            count += 1
            if count == 7:
                print()
                print("Failed to solve in 6 turns")
                print()
                print("Was it any of these words?: " + str(valid_guesses[0:5]))
                return

    if mode == "play":
        count = 1
        target = random.choice(answer_list)
        help = str(input("Do you want help?: 'Yes' or 'No': "))
        print()
        # iterate through user prompts
        while True:
            if help == "No":
                guess = input("GUESS " + str(count) + ", ENTER YOUR GUESS: ")
                print()
            if help == "Yes":
                guess = input("GUESS " + str(count) + ", RECOMMENDED: " + str(valid_guesses[0:5]) + ", ENTER YOUR GUESS: ")
                print()
            guess_letters = [letter for letter in guess]
            guess_colors = get_color(guess, target)
            letter_colors = list(zip(guess_letters, guess_colors))
            if guess_colors == [1, 1, 1, 1, 1]:
                print("Congrats you won in " + str(count) + " guesses")
                return
            print("The colors of your guess are: " + str(letter_colors))
            print()
            # reduce valid guess size recommendations
            valid_guesses = eliminate(valid_guesses, letter_colors)
            count += 1
            if count == 7:
                print("Failed to solve in 6 turns, the word was " + target.upper())
                return    

In [106]:
# Play a random game of wordle with the bot
solver("play")

Do you want help?: 'Yes' or 'No': Yes

GUESS 1, RECOMMENDED: ['soare', 'stare', 'roate', 'raile', 'arose'], ENTER YOUR GUESS: alter

The colors of your guess are: [('a', 0.5), ('l', 0.5), ('t', 0), ('e', 1), ('r', 0)]

GUESS 2, RECOMMENDED: ['samel', 'salep', 'lanes', 'lacey', 'panel'], ENTER YOUR GUESS: lanes

The colors of your guess are: [('l', 0.5), ('a', 1), ('n', 0), ('e', 1), ('s', 0)]

GUESS 3, RECOMMENDED: ['camel', 'paled', 'haled', 'baled', 'galed'], ENTER YOUR GUESS: camel

The colors of your guess are: [('c', 0), ('a', 1), ('m', 0), ('e', 1), ('l', 1)]

GUESS 4, RECOMMENDED: ['bagel', 'gavel', 'favel', 'hazel', 'babel'], ENTER YOUR GUESS: bagel

Congrats you won in 4 guesses


In [95]:
# Cheat through a real live wordle game
solver("cheat")


GUESS 1, RECOMMENDED: ['soare', 'stare', 'roate', 'raile', 'arose'], ENTER YOUR GUESS: alter
What are the colors? 1 for Green, .5 for Yellow, 0 for Gray: 0
What are the colors? 1 for Green, .5 for Yellow, 0 for Gray: 0
What are the colors? 1 for Green, .5 for Yellow, 0 for Gray: 0
What are the colors? 1 for Green, .5 for Yellow, 0 for Gray: .5
What are the colors? 1 for Green, .5 for Yellow, 0 for Gray: 0

GUESS 2, RECOMMENDED: ['noise', 'sonce', 'souce', 'shone', 'shine'], ENTER YOUR GUESS: binds
What are the colors? 1 for Green, .5 for Yellow, 0 for Gray: 0
What are the colors? 1 for Green, .5 for Yellow, 0 for Gray: 1
What are the colors? 1 for Green, .5 for Yellow, 0 for Gray: 1
What are the colors? 1 for Green, .5 for Yellow, 0 for Gray: 0
What are the colors? 1 for Green, .5 for Yellow, 0 for Gray: 0

GUESS 3, RECOMMENDED: ['yince', 'mince', 'wince', 'hinge', 'minge'], ENTER YOUR GUESS: wince
What are the colors? 1 for Green, .5 for Yellow, 0 for Gray: 1
What are the colors? 1 f

In [107]:
# See how the bot would automatically solve a given word
solver("auto")

What word would you like me to solve?: bagel

I guessed soare

I guessed alien

I guessed palet

I guessed lacey

I guessed bagel

The word is BAGEL, solved in 5 guesses
